In [26]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
import nltk
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dk980\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [27]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
410,I'll make this brief. This was a joy to watch....,positive
701,"Based on an actual story, John Boorman shows t...",positive
396,I picked up the movie with no cover and not ev...,positive
432,"My God, what an incredible movie it is! Remind...",negative
52,This is an EXCELLENT example of early Bette Da...,positive


In [28]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [29]:
df = normalize_text(df)
df.head()

,review,sentiment
410,make brief joy watch may may effective charact...,positive
701,based actual story john boorman show struggle ...,positive
396,picked movie cover even knowing wa watched lau...,positive
432,god incredible movie is reminded much similar ...,negative
52,excellent example early bette davis talent pro...,positive


In [30]:
df['sentiment'].value_counts()

sentiment
negative    255
positive    245
Name: count, dtype: int64

In [31]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [32]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
410,make brief joy watch may may effective charact...,1
701,based actual story john boorman show struggle ...,1
396,picked movie cover even knowing wa watched lau...,1
432,god incredible movie is reminded much similar ...,0
52,excellent example early bette davis talent pro...,1


In [33]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [34]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

In [36]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/Shreyansh19-o/Capstone-project.mlflow/')
dagshub.init(repo_owner='Shreyansh19-o', repo_name='Capstone-project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Initialized MLflow to track repo "Shreyansh19-o/Capstone-project"

2025-06-04 09:57:31,053 - INFO - Initialized MLflow to track repo "Shreyansh19-o/Capstone-project"


Repository Shreyansh19-o/Capstone-project initialized!

2025-06-04 09:57:31,060 - INFO - Repository Shreyansh19-o/Capstone-project initialized!


<Experiment: artifact_location='mlflow-artifacts:/bdb6e42618904546b94916be35c49070', creation_time=1749010730157, experiment_id='0', last_update_time=1749010730157, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [37]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.30)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-06-04 09:57:31,528 - INFO - Starting MLflow run...


2025-06-04 09:57:32,246 - INFO - Logging preprocessing parameters...
2025-06-04 09:57:33,599 - INFO - Initializing Logistic Regression model...
2025-06-04 09:57:33,600 - INFO - Fitting the model...
2025-06-04 09:57:33,646 - INFO - Model training complete.
2025-06-04 09:57:33,648 - INFO - Logging model parameters...
2025-06-04 09:57:34,084 - INFO - Making predictions...
2025-06-04 09:57:34,088 - INFO - Calculating evaluation metrics...
2025-06-04 09:57:34,115 - INFO - Logging evaluation metrics...
2025-06-04 09:57:35,989 - INFO - Saving and logging the model...
c:\Users\dk980\anaconda3\envs\atlas\lib\site-packages\_distutils_hack\__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure